In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Model construction

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU

In [102]:
def create_model(number_of_feat=1, height=100, width=100):
    model = Sequential()
    model.add(Conv2D(16, kernel_size=(3,3), input_shape=(number_of_feat, height, width, 4), padding='same')) 
    model.add(LeakyReLU(0.1))
    model.add(Conv2D(32, kernel_size=(3,3), padding='same'))  
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.25))
    model.add(Conv2D(32, kernel_size=(3,3), padding='same'))  
    model.add(LeakyReLU(0.1))
    model.add(Conv2D(64, kernel_size=(3,3), padding='same')) 
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(0.1))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation("relu"))
    return model

In [34]:
data = pd.read_csv("population.csv",encoding='latin1', sep="\t", header=None)
data.columns = ["year","city/county/municipality","Population number", "Age unknown", "Population aged 0-14", "Population aged 15-64", "Population aged 65+", "Dependancy ratio"]
print(data.shape)
data.head()

(4693, 8)


,year,city/county/municipality,Population number,Age unknown,Population aged 0-14,Population aged 15-64,Population aged 65+,Dependancy ratio
0,1989,Harju county,605415,0,129235,415937,60243,45.6
1,1989,..Keila city,10072,0,2474,6753,845,49.1
2,1989,..Loksa city,4315,0,1183,2877,255,50.0
3,1989,..Maardu city,16052,0,4346,10961,745,46.4
4,1989,..Paldiski city,7690,0,2005,5383,302,42.9


In [110]:
X = get_dependent_variable()
print(X.shape)
y = data.loc[data['city/county/municipality'] == '..Tallinn', 'Population number'].iloc[:1]
print(y)
#y = data.loc[data['city/county/municipality'] == 'Tallinn', 'Population number']
INIT_LR = 5e-3
BATCH_SIZE = 32
EPOCHS = 10

X = np.random.normal(1, 1, (4, 100, 100, 4))
y = np.random.normal(1, 1, 10)

model = create_model()
model.compile(
    loss='mse',
    optimizer=keras.optimizers.adamax(lr=INIT_LR), 
    metrics=['mse']
)
model.fit(X, y, batch_size=BATCH_SIZE,
    epochs=EPOCHS)
    

(1, 4, 100, 100, 4)
6    478974
Name: Population number, dtype: object


C:\Users\komisare\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\komisare\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':


ValueError: Input 0 is incompatible with layer conv2d_54: expected ndim=4, found ndim=5

In [49]:
data_cleaned = data.loc[data['Population number'].str.isdigit()]
y.shape

(4542,)

In [69]:
from scipy.misc import imread, imresize

def get_dependent_variable(cities=['Tallinn'], sources=['soil', 'ndvi', 'temperature', 'city_lights'],
                            img_height=100, img_width=100):
    X = np.zeros((len(cities), len(sources), img_height, img_width, 4))
    for i in range(len(cities)):
        city = cities[i]
        for j in range(len(sources)):
            source = sources[j]
            file = imread("data/" + city + "/" + source + ".png")
            img_array = np.array(file)
            img_array = imresize(img_array, (img_height, img_width))
            X[i, j, :, :, :] = img_array
    return X
            
X = get_dependent_variable()
X.shape
    

C:\Users\komisare\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\komisare\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':


(1, 4, 100, 100, 4)

In [72]:
y = data.loc[data['city/county/municipality'] == '..Tallinn', 'Population number']
y

6       478974
253     402390
500     400090
747     396490
994     395050
1241    398310
1488    400500
1735    398380
1982    396770
2229    396800
2476    397400
2723    398500
2970    401120
3217    403862
3464    406059
3711    411063
3958    413782
4205    423420
4452    426538
Name: Population number, dtype: object